In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
data=pd.read_csv('/content/gdrive/My Drive/dataset_7_features.csv')
data=np.array(data)
X=data[:,0]
y_label=data[:,1]
print(len(X))
X_train, X_test, Y_train, Y_test = train_test_split(X, y_label, test_size=0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train, test_size=0.1)

1018002


In [ ]:
train_texts=[str(s) for s in X_train]
test_texts=[str(s) for s in X_test]
val_texts=[str(s) for s in X_val]

In [ ]:
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)

# construct a new vocabulary
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)
val_texts = tk.texts_to_sequences(val_texts)

# Padding
train_data = pad_sequences(train_sequences,maxlen=75, padding='post')
test_data = pad_sequences(test_texts,maxlen=75,padding='post')
val_data = pad_sequences(val_texts,maxlen=75,padding='post')


# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')
val_data = np.array(val_data, dtype='float32')


In [ ]:
tk.word_index

{'0': 27,
 '1': 28,
 '2': 29,
 '3': 30,
 '4': 31,
 '5': 32,
 '6': 33,
 '7': 34,
 '8': 35,
 '9': 36,
 'UNK': 37,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
vocab_size=len(tk.word_index)
vocab_size

37

In [ ]:
n_domains=len(train_data)
seq_length=len(train_data[0])
train_X=np.reshape(train_data,(len(train_data),seq_length,1))
test_X=np.reshape(test_data,(len(test_data),seq_length,1))
val_X=np.reshape(val_data,(len(val_data),seq_length,1))

train_X.shape

(732960, 75, 1)

In [ ]:
embed=tf.keras.layers.Embedding(vocab_size+1,1, input_length=75)
train_eX=embed(train_data)
test_eX=embed(test_data)
val_eX=embed(val_data)

In [ ]:
train_eX.shape

TensorShape([732960, 75, 1])

In [ ]:
# one hot encodes the output variable
train_y = np_utils.to_categorical(Y_train)
test_y=np_utils.to_categorical(Y_test)
val_y=np_utils.to_categorical(Y_val)


In [ ]:
#model 1
embedding_dim =64
max_length =75
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy', 'AUC'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 75, 64)            2368      
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 68,469
Trainable params: 68,469
Non-trainable params: 0
_________________________________________________________________


In [4]:
#model 2
model2 = tf.keras.Sequential([
   # tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(75)),
   # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy', 'AUC'])

model2.build(input_shape=(None,75,1))
model2.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 150)              46200     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                9664      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 55,994
Trainable params: 55,994
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=model2.fit(train_eX, train_y,validation_data=(val_eX,val_y) ,epochs = 10, batch_size=128)

In [ ]:
model.save('/content/gdrive/My Drive/models/lstm2_model2_50.h5')

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])


plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)


In [ ]:
test_X=np.reshape(test_data,(len(test_data),63,1))
test_X.shape
predictions=model.predict(test_X)

In [ ]:
test_X = np.asarray(test_X).astype('float32')
test_y=np.asarray(test_y).astype('float32')
test_loss, test_acc = model.evaluate(test_X,test_y)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


In [ ]:
model = tf.keras.models.load_model('/content/gdrive/My Drive/models/lstm2_n.h5')

FGSM attack:

In [ ]:
def fast_gradient_method(
    model_fn,
    x,
    eps,
    norm,
    loss_fn=None,
    clip_min=None,
    clip_max=None,
    y=None,
    targeted=False,
    iterative=False,
    sanity_checks=False,
):

    if y is None:
        y=np.argmax(model(x))

    if loss_fn is None:
      loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

    if iterative:#if iterative, give also iters
      adv_X,optimal_perturbation=iterative_fgsm(model_fn,x,y,targeted,iters)

    
    #cast if numpy array is given
    x = tf.cast(x, tf.float32)

    #function to compute gradients
    grad = compute_gradient(model_fn, loss_fn, x, y, targeted,iters=1)
    
    #function to find optimal perturbation based on norm
    optimal_perturbation=norm_constraint(grad,eps,norm)
    #adv image created
    adv_x=x+optimal_perturbation

    norm=l2(adv_x,x)
      
    preds=model.predict(adv_x)
    lables=np.argmax(preds,axis=1)
    print(preds,lables)
      
    confidence=np.max(preds,axis=1)*100
    print(confidence)


    return adv_x


In [ ]:
def compute_gradient(model,loss_fn,x,y,targeted,iters):
    for epoch in range(iters):
      with tf.GradientTape() as gt:
        gt.watch(x)
        label=model(x)
        loss=loss_fn(y,label)
        #print(loss)
        if(targeted):
          loss=-loss

    grad=gt.gradient(loss,x)
    #print(grad)
    return grad

In [ ]:
def norm_constraint(grad, eps, norm=np.inf):
    """
    Solves for the optimal input to a linear function under a norm constraint.
    Optimal_perturbation = argmax_{eta, ||eta||_{norm} < eps} dot(eta, grad)
    :param grad: tf tensor containing a batch of gradients
    :param eps: float scalar specifying size of constraint region
    :param norm: int specifying order of norm
    :returns:
      tf tensor containing optimal perturbation
    """

    # Convert the iterator returned by `range` into a list.
    axis = list(range(1, len(grad.get_shape())))
    avoid_zero_div = 1e-12
    if norm == np.inf:
        # Take sign of gradient
        optimal_perturbation = tf.sign(grad)
        # The following line should not change the numerical results. It applies only because
        # `optimal_perturbation` is the output of a `sign` op, which has zero derivative anyway.
        # It should not be applied for the other norms, where the perturbation has a non-zero derivative.
        optimal_perturbation = tf.stop_gradient(optimal_perturbation)
    elif norm == 1:
        abs_grad = tf.abs(grad)
        sign = tf.sign(grad)
        max_abs_grad = tf.reduce_max(abs_grad, axis, keepdims=True)#maximum value of gradient
        tied_for_max = tf.dtypes.cast(
            tf.equal(abs_grad, max_abs_grad), dtype=tf.float32
        )
        num_ties = tf.reduce_sum(tied_for_max, axis, keepdims=True)
        optimal_perturbation = sign * tied_for_max / num_ties
    elif norm == 2:
        square = tf.maximum(
            avoid_zero_div, tf.reduce_sum(tf.square(grad), axis, keepdims=True)
        )
        optimal_perturbation = grad / tf.sqrt(square)
    else:
        raise NotImplementedError(
            "Only L-inf, L1 and L2 norms are currently implemented."
        )

    # Scale perturbation to be the solution for the norm=eps rather than norm=1 problem
    scaled_perturbation = tf.multiply(eps, optimal_perturbation)
    return scaled_perturbation

In [ ]:
def l2(x, y):
  # technically squarred l2
    return tf.reduce_sum(tf.square(x - y), list(range(1, len(x.shape))))

In [ ]:
rev_ind= dict((tk.word_index[k], k) for k in tk.word_index)
def convert_to_domain(adv_x,vocab_size,word_ind):
  domain=[]
  inds=[]
  adv_x=np.asarray(adv_x).reshape(1,75).flatten()

  scale=lambda ind : ind*vocab_size if ind < 1 else ind
  abs_inds=abs(adv_x)
  inds=[round(scale(ind)) for ind in abs_inds]
  domains=[rev_ind[ind] if ind!=0 else 0 for ind in inds]
  dom=""
  for c in domains:
    if c!=0:
      dom+=c
  print(dom)
  return dom



In [ ]:
x= tk.texts_to_sequences(['sogupteabok'])
#x= tk.texts_to_sequences(['google'])
x=pad_sequences(x,maxlen=75, padding='post')
x=np.array(x)
x=x.reshape(1,75)
embed=tf.keras.layers.Embedding(vocab_size+1,1, input_length=75)
x=embed(x)
print(x.shape)
y=np.array([1,0]).reshape(1,2)
print(y)

(1, 75, 1)
[[1 0]]


In [ ]:
label=model(x)
np.argmax(label)

0

In [ ]:
loss_fn=tf.keras.losses.CategoricalCrossentropy()
eps=0.7
t_lable=np.array([0,1]).reshape(1,2)
adv_x=fast_gradient_method(model,x,eps,2,clip_min=-1,clip_max=1,y=t_lable,targeted=True,loss_fn=loss_fn)

adv_domain=convert_to_domain(adv_x,vocab_size,rev_ind)

[[0.42085698 0.579143  ]] [1]
[57.9143]
cfbbjcbaaiccaaaaaaaaaabbbbbbbbbbbbbbccccccccccccccccccdddddeeeeeeedddddcb


In [ ]:
#FGSM training
cnt=0
adv_train_eX=[]
for i in range(len(train_eX)):

  x=train_eX[i]
  x=np.asarray(x)
  x=x.reshape(1,75,1)
  t_lable=np.abs(1-train_y[i]).reshape(1,2)
  eps=0.7
  loss_fn=tf.keras.losses.CategoricalCrossentropy()
  print(x.shape)
  adv_x=fast_gradient_method(model,x,eps,2,clip_min=-1,clip_max=1,y=t_lable,targeted=True,loss_fn=loss_fn)
  
  preds=model.predict(adv_x)
  lables=np.argmax(preds,axis=1)
  print(Y_train[i],lables[0])
  if Y_train[i]!=lables[0]:
    adv_train_eX.append(adv_x)
    cnt=cnt+1
  else:
    print('attack fail')

adv_train_eX=np.array(adv_train_eX)
print(adv_train_eX.shape)
print(cnt)
